### Notebook - 1b

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',95)
pd.set_option('display.max_rows',None)
import pickle

In [ ]:
# f = open('df1.pkl', 'wb')
# pickle.dump(r, f)
# f.close()

In [2]:
f = open('df1.pkl', 'rb')
r = pickle.load(f)
f.close()

### Category Cleaning
In order to analyze liquor sales, attention must be given to how each liquor order has been categorized. Looking at the original data set, we see there are 133 unique category names which is too long a list to make sense of so we are going to re-classify the category_name into 13 simplified liquor categories (ie. rum, whiskey, gin, brandy, vodka, etc..) so we can have a clearer, more top-line overview of the types of liquor being ordered/sold.

In [3]:
# Making a copy of the original dataframe to work with the category features
liquor_cat = r.copy()

In [ ]:
# Looking at the category_names 
# liquor_cat['category_name'].value_counts()

In [4]:
liquor_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20095649 entries, 0 to 20095648
Data columns (total 11 columns):
 #   Column               Dtype         
---  ------               -----         
 0   invoice/item_number  category      
 1   date                 datetime64[ns]
 2   store_number         int64         
 3   category             float64       
 4   category_name        object        
 5   item_number          category      
 6   item_description     category      
 7   bottles_sold         object        
 8   sale_dollars         object        
 9   volume_sold_liters   float16       
 10  store_name           category      
dtypes: category(4), datetime64[ns](1), float16(1), float64(1), int64(1), object(3)
memory usage: 2.0+ GB


In [5]:
#liquor_cat.loc[liquor_cat.category_name == 'NaN'] 
category_null = liquor_cat[liquor_cat['category_name'].isna()]

In [6]:
# Cleaning item_description so object is in all lower case letters 
category_null['item_description'] = category_null['item_description'].str.lower()

In [7]:
category_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25040 entries, 2389 to 20093545
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   invoice/item_number  25040 non-null  category      
 1   date                 25040 non-null  datetime64[ns]
 2   store_number         25040 non-null  int64         
 3   category             8066 non-null   float64       
 4   category_name        0 non-null      object        
 5   item_number          25040 non-null  category      
 6   item_description     25040 non-null  object        
 7   bottles_sold         25040 non-null  object        
 8   sale_dollars         25040 non-null  object        
 9   volume_sold_liters   24593 non-null  float16       
 10  store_name           25040 non-null  category      
dtypes: category(3), datetime64[ns](1), float16(1), float64(1), int64(1), object(4)
memory usage: 795.6+ MB


In [8]:
category_null.head() 

,invoice/item_number,date,store_number,category,category_name,item_number,item_description,bottles_sold,sale_dollars,volume_sold_liters,store_name
2389,S06688800087,2012-07-19,2517,1022200.0,NaN,3657,herradura gold reposado 6pak,2,67.78,1.50,Hy-Vee Food Store #1 / Newton
3816,S21839300104,2014-10-16,2633,1022200.0,NaN,3657,herradura gold reposado 6pak,6,212.22,4.50,Hy-Vee #3 / BDI / Des Moines
4414,S05816100002,2012-05-31,2599,1022200.0,NaN,904983,herradura anejo tequila,6,243.18,4.50,Hy-Vee Wine and Spirits / Coralville
4757,S27709000001,2015-10-15,5063,NaN,NaN,1488,grey goose w/2 martini glasses,3,68.91,2.25,Logan Convenience Store
6836,S21550000040,2014-10-01,2551,NaN,NaN,3005,chambord liqueur w/2 glasses,6,167.04,4.50,Hy-Vee Food Store / Chariton


In [9]:
category_null.category_name = category_null.category_name.fillna('') 

In [10]:
category_null.head()

,invoice/item_number,date,store_number,category,category_name,item_number,item_description,bottles_sold,sale_dollars,volume_sold_liters,store_name
2389,S06688800087,2012-07-19,2517,1022200.0,,3657,herradura gold reposado 6pak,2,67.78,1.50,Hy-Vee Food Store #1 / Newton
3816,S21839300104,2014-10-16,2633,1022200.0,,3657,herradura gold reposado 6pak,6,212.22,4.50,Hy-Vee #3 / BDI / Des Moines
4414,S05816100002,2012-05-31,2599,1022200.0,,904983,herradura anejo tequila,6,243.18,4.50,Hy-Vee Wine and Spirits / Coralville
4757,S27709000001,2015-10-15,5063,NaN,,1488,grey goose w/2 martini glasses,3,68.91,2.25,Logan Convenience Store
6836,S21550000040,2014-10-01,2551,NaN,,3005,chambord liqueur w/2 glasses,6,167.04,4.50,Hy-Vee Food Store / Chariton


In [11]:
# Creating and cleaning the category names into broader categories

def contains(string, match_list):
    for match in match_list:
        if match in string:
            return True
    return False

def item_cleaner(value):
    """
    To be applied to a dataframe column to clean
    alcohol categories so they are simpler.
    """
    import pandas as pd
    
    if contains(value, ['rum', 'rums', 'bacardi', 'cruzan', 'abuelo', 'morgan']):
        return 'rum'
    elif contains(value, ['whiskies','whiskey', 'whisky', 'scotch', 'rye', 'bourbon', 'iowa distilleries', 'fireball', 'mark', 'ha', 'medley', 'forester', 'jack', 'woodford']):
        return 'whiskey'
    elif contains(value, ['anisette', 'triple sec', 'amaretto', 'creme', 'cremes', 'creams', 'cream', 'cordials', 'cordial', 'liqueurs', 'liqueur', '99', 'marnier']):
        return 'cordials liqueurs'
    elif contains(value, ['brandy','brandies', 'cognac', 'coganc', 'hennessy']):
        return 'brandy'
    elif contains(value, ['tequila','tequilas', 'mezcal', 'hornitos', 'herradura']):
        return 'tequila'
    elif contains(value, ['schnapps','schnapp']):
        return 'schnapps'
    elif contains(value, ['vodka','vodkas','eddy', 'burnett\'s', 'burnetts', 'wapsi', 'smirnoff', 'cane']):
        return 'vodka'
    elif contains(value, ['gin','gins']):
        return 'gin'
    elif contains(value, ['cocktails','cocktail']):
        return 'cocktails'
    elif contains(value, ['spirit','spirits']):
        return 'spirits'
    elif contains(value, ['american alcohol', 'everclear']):
        return 'grain alcohol'
    elif contains(value, ['decanters', 'decanter', 'packages', 'glasses', 'coaster', 'shot', '/flask', 'flask', 'w/glass']):
        return 'accessories'
    else:
        return 'other'
    
category_null['category_name'] = category_null['item_description'].apply(item_cleaner)

In [12]:
category_null.head()

,invoice/item_number,date,store_number,category,category_name,item_number,item_description,bottles_sold,sale_dollars,volume_sold_liters,store_name
2389,S06688800087,2012-07-19,2517,1022200.0,tequila,3657,herradura gold reposado 6pak,2,67.78,1.50,Hy-Vee Food Store #1 / Newton
3816,S21839300104,2014-10-16,2633,1022200.0,tequila,3657,herradura gold reposado 6pak,6,212.22,4.50,Hy-Vee #3 / BDI / Des Moines
4414,S05816100002,2012-05-31,2599,1022200.0,tequila,904983,herradura anejo tequila,6,243.18,4.50,Hy-Vee Wine and Spirits / Coralville
4757,S27709000001,2015-10-15,5063,NaN,accessories,1488,grey goose w/2 martini glasses,3,68.91,2.25,Logan Convenience Store
6836,S21550000040,2014-10-01,2551,NaN,whiskey,3005,chambord liqueur w/2 glasses,6,167.04,4.50,Hy-Vee Food Store / Chariton


In [13]:
category_null.category_name.value_counts() 

other                5269
whiskey              3976
cordials liqueurs    3586
brandy               2678
rum                  2481
vodka                2387
tequila              2374
accessories          2022
gin                   210
grain alcohol          48
schnapps                6
spirits                 3
Name: category_name, dtype: int64

In [14]:
# Fill in missing values in liquor_cat table's category_name column with empty string
liquor_cat.category_name = liquor_cat.category_name.fillna('')

In [15]:
category_null.drop(columns = ['date', 'store_number', 'category', 'item_number', 'item_description','bottles_sold', 
                    'sale_dollars', 'volume_sold_liters','store_name'], axis = 1, inplace = True)

In [16]:
category_null.columns

Index(['invoice/item_number', 'category_name'], dtype='object')

In [26]:
# liquor_cat = pd.merge(liquor_cat, category_null, how = 'left', on = 'invoice/item_number')

KeyboardInterrupt: 

In [18]:
# liquor_cat.head()

,invoice/item_number,date,store_number,category,category_name_x,item_number,item_description,bottles_sold,sale_dollars,volume_sold_liters,store_name,category_name_y
0,S04763500007,2012-03-27,2534,1012100.0,CANADIAN WHISKIES,11788,Black Velvet,6,94.02,10.50,Hy-Vee Drugtown / Urbandale,NaN
1,S27474100012,2015-08-25,4924,1022100.0,TEQUILA,89194,Jose Cuervo Especial Reposado Flask,4,33.00,1.50,Abby Lea's,NaN
2,S10731000040,2013-02-21,4652,1032200.0,IMPORTED VODKA - MISC,34449,Ketel One Citroen,2,40.48,1.50,Brady Mart Food & Liquor,NaN
3,S17037900080,2014-01-27,4794,1041100.0,AMERICAN DRY GINS,32236,Seagrams Extra Dry Gin,1,8.99,0.75,Smokin' Joe's #17 Tobacco and Liquor Outlet,NaN
4,S14396900023,2013-09-09,2647,1012100.0,CANADIAN WHISKIES,13038,Canadian Reserve Whisky,6,80.94,10.50,Hy-Vee #7 / Edgewood Cedar Rapids,NaN


In [23]:
liquor_cat.loc[liquor_cat['invoice/item_number'].isin(category_null['invoice/item_number']), ['category_name_x']] =category_null['category_name']

In [24]:
liquor_cat.loc[liquor_cat['invoice/item_number'].isin(category_null['invoice/item_number']), ['category_name_x']].head()

,category_name_x
2389,tequila
3816,tequila
4414,tequila
4757,accessories
6836,whiskey


In [22]:
category_null.head()

,invoice/item_number,category_name
2389,S06688800087,tequila
3816,S21839300104,tequila
4414,S05816100002,tequila
4757,S27709000001,accessories
6836,S21550000040,whiskey


In [25]:
# S06688800087
liquor_cat[liquor_cat['invoice/item_number'] == 'S06688800087']

,invoice/item_number,date,store_number,category,category_name_x,item_number,item_description,bottles_sold,sale_dollars,volume_sold_liters,store_name,category_name_y
2389,S06688800087,2012-07-19,2517,1022200.0,tequila,3657,Herradura Gold Reposado 6pak,2,67.78,1.5,Hy-Vee Food Store #1 / Newton,tequila


In [ ]:
# Cleaning category_name so object is in all lower case letters
liquor_cat['category_name'] = liquor_cat['category_name'].str.lower()

In [ ]:
# next steps: re-classify below to get the corrected 'category_name' column
# then create the egg nog column
# then need the store closed column 

### Methodology for re-classifying liquor categories
As mentioned above, there are 133 category_names, to bucket them into a broader category we created 2 functions,

1) the "contains" function which will look at every category_name and see if there is a string match and 2) the "category_cleaner" function which will replace the value of the category_name with a given word if there is a match from the contains function.

In [ ]:
# Creating and cleaning the category names into broader categories

def contains(string, match_list):
    for match in match_list:
        if match in string:
            return True
    return False

def category_cleaner(value):
    """
    To be applied to a dataframe column to clean
    alchol categories so they are simpler.
    """
    import pandas as pd
    
    if pd.isnull(value):
        return value
    elif contains(value, ['rum', 'rums']):
        return 'rum'
    elif contains(value, ['whiskies','whiskey', 'whisky', 'scotch', 'rye', 'bourbon', 'iowa distilleries']):
        return 'whiskey'
    elif contains(value, ['anisette', 'triple sec', 'amaretto', 'creme', 'cremes', 'creams', 'cream', 'cordials', 'cordial', 'liqueurs', 'liqueur']):
        return 'cordials liqueurs'
    elif contains(value, ['brandy','brandies']):
        return 'brandy'
    elif contains(value, ['tequila','tequilas', 'mezcal']):
        return 'tequila'
    elif contains(value, ['schnapps','schnapp']):
        return 'schnapps'
    elif contains(value, ['vodka','vodkas']):
        return 'vodka'
    elif contains(value, ['gin','gins']):
        return 'gin'
    elif contains(value, ['cocktails','cocktail']):
        return 'cocktails'
    elif contains(value, ['spirit','spirits']):
        return 'spirits'
    elif contains(value, ['american alcohol']):
        return 'grain alcohol'
    elif contains(value, ['decanters', 'decanter', 'packages']):
        return 'accessories'
    else:
        return 'special orders'
    
liquor_cat['cat_name2'] = liquor_cat['category_name'].apply(category_cleaner)

In [ ]:
liquor_cat['cat_name2'].value_counts()

In [ ]:
liquor_cat.columns.tolist()

In [ ]:
liquor_cat.shape

In [ ]:
liquor_cat.head()

In [ ]:
liquor_cat = liquor_cat.drop('category_name', axis=1)

In [ ]:
liquor_cat = liquor_cat.rename(columns= {'store_name_y':'store_name2','cat_name2':'category_name2'})

In [ ]:
liquor_cat.drop('category',axis=1,inplace=True)

In [ ]:
liquor_cat.columns

In [ ]:
# This is our original table with correct store names and saving to csv so we don't have to re-load
liquor_cat.to_csv('./data/main_df2.csv.gz',index=False,compression='gzip')